In [ ]:
import multiprocessing
import sys
import threading
from time import sleep
from tqdm import tqdm

# from tqdm.notebook import tqdm
from functools import partial


def count_test(pool_size, pool_index, lock):
    print(pool_index)
    with lock:
        print(" ", end="", flush=True)
        # self.pbar = tqdm(desc="Parsing pbf file", total=self.features_count, position=self.pool_index)
        pbar = tqdm(desc=f"Parsing pbf file #{pool_index}", total=10, position=pool_index)

    for _ in tqdm(range(10), position=pool_index):
        sleep(1)
        with lock:
            pbar.update(n=1)

    with lock:
        pbar.close()


pool_size = 3
pool = multiprocessing.Pool(pool_size, initializer=tqdm.set_lock, initargs=(tqdm.get_lock(),))

lock = multiprocessing.Manager().Lock()

# pbar = tqdm(desc="Parsing pbf file", total=10)

for pool_index in range(pool_size):
    # f = partial(count_test, pbar)
    pool.apply_async(count_test, [pool_size, pool_index, lock])
    # f()
    # pool.apply_async(f)
pool.close()
pool.join()

# pbar.close()

In [ ]:
import time
from tqdm import tqdm
import multiprocessing as mp


def test(queue, bar_queue):
    while True:
        x = queue.get()
        if x is None:
            break
        for _ in range(x):
            time.sleep(0.05)
            bar_queue.put_nowait(1)


def update_bar(q):
    pbar = tqdm(total=188)

    while True:
        x = q.get()
        pbar.update(x)


if __name__ == "__main__":
    queue = mp.Queue()
    bar_queue = mp.Queue()

    processes = [mp.Process(target=test, args=(queue, bar_queue)) for _ in range(4)]

    # start update progress bar process
    # daemon= parameter is set to True so this process won't block us upon exit
    bar_process = mp.Process(target=update_bar, args=(bar_queue,), daemon=True)
    bar_process.start()

    for p in processes:
        p.start()

    for idx in range(20):
        queue.put(idx)

    for _ in range(4):
        queue.put(None)  # sentinel values to signal subprocesses to exit

    for p in processes:
        p.join()  # wait for all subprocesses to finish

    time.sleep(0.5)